In [1]:
!pip install sklearn pandas
!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
import json
import pandas as pd
with open("../scripts/datascraping/data/flyers_text.json", 'r') as file:
    data = json.load(file)

docs = []
df = pd.DataFrame(data.items(), columns=['Title', 'Document'])
df["Document"] = df["Document"].astype(str)
    
df

,Title,Document
0,(Langdurig) gebroken vliezen,{'(Langdurig) gebroken vliezen': 'Als de vliez...
1,Aambeien (heamorroiden),{'Aambeien': '(heamorroiden) Deze folder geeft...
2,Aangezichtsverlamming,{'Aangezichtsverlamming': 'Deze folder heeft t...
3,Aangifte van geboorte,{'Aangifte geboorte': 'Binnen 3 dagen geeft u ...
4,Aanvraagformulier kopie medische gegevens,{'Aanvraagformulier kopie medische gegevens': ...
...,...,...
1315,Zwangerschapscontroles,"{'Zwangerschapscontroles': 'Gefeliciteerd, u b..."
1316,Zwangerschapsdiabetes en voeding,{'Zwangerschapsdiabetes en voeding': 'Uw medis...
1317,Zwangerschapsdiabetes,{'Zwangerschapsdiabetes': 'Uw medisch speciali...
1318,Zwelling van de balzak,{'Zwelling van de balzak': 'Binnenkort wordt u...


In [13]:
import pandas as pd
from stem import stem_sentence
from tqdm.notebook import tqdm

# Assuming df is your existing DataFrame and it has a column named 'Document'

# Create an empty list to store stemmed documents
stemmed_docs = []

# Loop through each document in the 'Document' column, stem it, and append to the list
for doc in tqdm(df['Document'], desc="Stemming"):
    stemmed_doc = stem_sentence(doc)
    stemmed_docs.append(stemmed_doc)

# Add the stemmed_docs as a new column to the existing DataFrame
df['Stemmed_Doc'] = stemmed_docs

# Display the first few rows of the DataFrame
df.head()


Stemming:   0%|          | 0/1320 [00:00<?, ?it/s]

,Title,Document,Stemmed_Doc
0,(Langdurig) gebroken vliezen,{'(Langdurig) gebroken vliezen': 'Als de vliez...,langdur gebrok vliez als vliez gebrok zwangers...
1,Aambeien (heamorroiden),{'Aambeien': '(heamorroiden) Deze folder geeft...,aambei heamorroid dez folder geeft global over...
2,Aangezichtsverlamming,{'Aangezichtsverlamming': 'Deze folder heeft t...,aangezichtsverlamm dez folder doel informatie ...
3,Aangifte van geboorte,{'Aangifte geboorte': 'Binnen 3 dagen geeft u ...,aangift geboort binn 3 dag geeft geboort baby ...
4,Aanvraagformulier kopie medische gegevens,{'Aanvraagformulier kopie medische gegevens': ...,aanvraagformulier kopie medisch gegeven let le...


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'df' is your existing DataFrame and it has columns 'Title' and 'Document'

# Create the TF-IDF model
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Document'])

# Convert the X matrix to a DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Set the 'Title' column as the index of the original DataFrame
df.set_index('Title', inplace=True)

# Concatenate the TF-IDF DataFrame with the original DataFrame (which now has 'Title' as index)
final_df = pd.concat([df, tfidf_df], axis=1)

# Write the final DataFrame to a CSV file, with 'Title' as the index
final_df.to_csv('tfidf_matrix_with_titles.csv')
